#Import packages

In [1]:
from xml.dom import minidom
from sys import stdin
from urllib import request
from subprocess import call
import numpy as np
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



#initialize user-defined variables

In [2]:
date = "2020/01/01"
site = "KCRP"
bucketURL = "http://noaa-nexrad-level2.s3.amazonaws.com"
dirListURL = bucketURL+ "/?prefix=" + date + "/" + site

#list available files

In [3]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

print ("listing files from %s" % dirListURL)

#xmldoc = minidom.parse(stdin)
xmldoc = minidom.parse(request.urlopen(dirListURL))
itemlist = xmldoc.getElementsByTagName('Key')
print (len(itemlist) , "keys found...")

for x in itemlist:
    file = getText(x.childNodes)
    #print ("Found %s " % file)

listing files from http://noaa-nexrad-level2.s3.amazonaws.com/?prefix=2020/01/01/KCRP
257 keys found...


#read NEXRAD radar file -- having issue reading from http, temporarily reading from local directory

In [4]:
#https://s3.amazonaws.com/noaa-nexrad-level2/index.html#2020/01/01/KCRP/
#https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/KCRP20200101_000431_V06

#radar = pyart.io.read('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
radar = pyart.io.read('KCRP20200101_000431_V06')
radar.info()

altitude:
	data: <ndarray of type: float64 and shape: (1,)>
	long_name: Altitude
	standard_name: Altitude
	units: meters
	positive: up
altitude_agl: None
antenna_transition: None
azimuth:
	data: <ndarray of type: float64 and shape: (6480,)>
	units: degrees
	standard_name: beam_azimuth_angle
	long_name: azimuth_angle_from_true_north
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (6480,)>
	units: degrees
	standard_name: beam_elevation_angle
	long_name: elevation_angle_from_horizontal_plane
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	differential_reflectivity:
		data: <ndarray of type: float32 and shape: (6480, 1832)>
		units: dB
		standard_name: log_differential_reflectivity_hv
		long_name: log_differential_reflectivity_hv
		valid_max: 7.9375
		valid_min: -7.875
		coordinates: elevation azimuth range
		_FillValue: -9999.0
	differenti

#confirm station location

In [5]:
altitude0 = radar.altitude['data'][0]
latitude0 = radar.latitude['data'][0]
longitude0 = radar.longitude['data'][0]
print(altitude0, latitude0, longitude0)

43.0 27.78401756286621 -97.5112533569336


In [40]:
gate_lat = radar.gate_latitude['data']
gate_lon = radar.gate_longitude['data']
gate_alt = radar.gate_altitude['data']
#len(radar.gate_latitude['data'])
#print(gate_lat)


#flat_list = [item for sublist in gate_lat for item in sublist]
#print(flat_list)

import itertools
merged = list(itertools.chain.from_iterable(gate_lat))
print(merged)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
import pandas as pd
#gate_lla = zip(gate_lat, gate_lon, gate_alt)
gate_lla = pd.DataFrame(
    {'Gate latitude': gate_lat,
     'Gate longitude': gate_lon,
     'Gate altitude': gate_alt
    })

ValueError: If using all scalar values, you must pass an index

In [ ]:
#sample_lat_dict = {'data': sample_lat, 'units': 'test', 'long_name': 'sample_latitude',
             '_FillValue': sample_lat.fill_value, 'standard_name': 'sample_latitude'}

#radar.add_field('sample_latitude', sample_lat_dict, replace_existing=True)

In [8]:
import netCDF4
import xarray as xr
pyart.io.write_cfradial('new_radar.nc', radar)

AttributeError: 'cftime._cftime.DatetimeGregorian' object has no attribute 'isoformat'

In [9]:
ds = xr.open_dataset('./new_radar.nc')
ds = ds[['reflectivity','velocity']]

df = ds.to_dataframe()
print(len(df))
print(df.columns) 
print(df)

11871360
Index(['reflectivity', 'velocity', 'azimuth', 'elevation'], dtype='object')
                                        reflectivity  velocity     azimuth  \
range    time                                                                
2125.0   2020-01-01 00:04:31.182999999          -8.0       NaN   16.254272   
         2020-01-01 00:04:31.278999999         -11.5       NaN   16.748657   
         2020-01-01 00:04:31.379000000         -15.0       NaN   17.248535   
         2020-01-01 00:04:31.475000000         -13.0       NaN   17.748413   
         2020-01-01 00:04:31.576999999         -14.0       NaN   18.248291   
...                                              ...       ...         ...   
459875.0 2020-01-01 00:11:18.696000000           NaN       NaN  213.549500   
         2020-01-01 00:11:18.751999999           NaN       NaN  214.549255   
         2020-01-01 00:11:18.807000000           NaN       NaN  215.549011   
         2020-01-01 00:11:18.860000000           NaN     